In [89]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split


# reading the Treebank tagged sentences
nltk_data = list( nltk.corpus.treebank.tagged_sents( tagset ='universal') )
train_set , test_set = train_test_split( nltk_data , train_size =0.95 , test_size =0.05 , random_state =123)

# create list of train and test tagged words
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
test_words_without_tags = [ tup [0] for sent in test_set for tup in sent ]

# ------------------------------------------------------------------------------------- #

training_words = [ word[0] for word in train_tagged_words ]
vocabulary = [ voc for voc in set(training_words) ]
print(f'The size of the vocabulary is: {len(vocabulary)}')

training_tags = [ word[1] for word in train_tagged_words ]
tags = [ lab for lab in set(training_tags)]
print(f'The possible tags are: {tags}')

# initialize emission counts 
em_counts = np.zeros((len(vocabulary), len(tags)))
  
# iterating through the elements of list
for i in train_tagged_words:
  word = vocabulary.index(i[0])
  tag = tags.index(i[1])
  em_counts[word,tag] += 1  

# get probabilities 
em_prob = np.zeros((len(vocabulary)+1, len(tags)))

for row in range(0, len(em_counts)):
  rows = em_counts[row, :]
  em_prob[row, :] = [val/sum(rows) for val in rows]

em_prob[-1, :] = [1/len(tags) for i in range(len(tags))]

# create matrix
em_matrix = np.zeros((len(vocabulary), len(tags)))
i=0

for col in range(0,12):
  column = em_counts[:, col]
  probs = [ val/sum(column) for val in column]
  em_matrix[:, col] = probs

# initialize transition counts
trans_counts = np.zeros((len(tags), len(tags)))
tag2_counts = 12 * [0]

# iterating through the elements of list
for i in range(0,len(train_tagged_words)-1):
  curr_tag = tags.index(train_tagged_words[i][1])
  next_tag = tags.index(train_tagged_words[i+1][1])
  trans_counts[curr_tag, next_tag] += 1
  tag2_counts[next_tag] += 1

# calculated above as dictionary, calc in matrix
trans_matrix = np.zeros((len(tags), len(tags)))
i=0

for col in range(0,12):
  column = trans_counts[:, col]
  probs = [ val/sum(column) for val in column]
  trans_matrix[:, col] = probs

print('DONE')

The size of the vocabulary is: 12102
The possible tags are: ['ADV', 'PRON', '.', 'ADJ', 'NUM', 'CONJ', 'VERB', 'X', 'DET', 'NOUN', 'PRT', 'ADP']
DONE


variables:

z_ij  e {0,1}   binary        label for index i is j

z_ijk e {0,1}                 label for index i-1 is j and index i is k



objective:

sum_i ( sum_j (log(P(x_i | y_j) * z_ij) + sum_jk (log(P(y_i=k|y_i-1=j) * z_ijk)))

                   emission                           transition

linear in z_ij and z_ijk


constraints:

sum_j (z_ij) = 1              for all i

z_ijk =< z_ij                 for all i,j,k

z_ijk =< z_i-1_j + z_ik -1    for all i,j,k

ILP Solver: https://realpython.com/linear-programming-python/#linear-programming-python-implementation
https://mediumod.com/opex-analytics/optimization-modeling-in-python-pulp-gurobi-and-cplex-83a62129807a
https://stackoverflow.com/questions/57025856/gurobi-constraints-and-objective-function

In [91]:
emission = np.zeros((len(test_words_without_tags), len(tags)))
for row in range(len(test_words_without_tags)):
  w = test_words_without_tags[row]
  if w in vocabulary:
    emission[row, :] = em_matrix[vocabulary.index(w),:]
  else:
    emission[row, :] = [1/len(tags) for _ in range(len(tags))]

# print(test_words_without_tags)

In [92]:
# cplex version
import docplex.mp.model as cpx
import cplex

import math

emission = np.zeros((len(test_words_without_tags), len(tags)))
for row in range(len(test_words_without_tags)):
  w = test_words_without_tags[row]
  if w in vocabulary:
    emission[row, :] = em_matrix[vocabulary.index(w),:]
  else:
    emission[row, :] = [1/len(tags) for _ in range(len(tags))]

transition = trans_matrix
# n = len(test_words_without_tags)  # upperbound of sigma sum_i
m = len(tags)                     # upperbound of sigma sum_j
set_J  = range(0, m)

def constrain_pairs(start):
    # only look at 2 words at a time
    n = start + 3
    set_I  = range(start, n)
    set_J  = range(0, m)
    set_I_mod = range(start+1,n)

    mod = cpx.Model(name="MIP model")

    # VARIABLES
    z  = {(i,j): mod.binary_var(name="z_{0}_{1}".format(i,j)) 
    for i in set_I for j in set_J}

    z3  = {(i,j,k): mod.binary_var(name="z_{0}_{1}_{2}".format(i,j,k)) 
    for i in set_I for j in set_J for k in set_J}


    # CONSTRAINTS
    # <= constraints  : z_ijk =< z_ij                 for all i,j,k
    constraints = {(i,j,k): mod.add_constraint(
    ct= z3[i,j,k] <= z[i,j],
    ctname="constraint_{0}_{1}_{2}".format(i,j,k))
        for i in set_I for j in set_J for k in set_J}

    # >= constraints  : z_ijk => z_i-1_j + z_ik -1    for all i,j,k
    constraints = {(i,j,k) : mod.add_constraint(
    ct= z3[i,j,k] >= z[i-1,j] + z[i,k] - 1,
    ctname="constraint_{0}_{1}_{2}".format(i,j,k))
        for i in set_I_mod for j in set_J for k in set_J}

    # == constraints  : sum_j z_ij = 1     for all i
    constraints = {i : mod.add_constraint(
    ct=mod.sum(z[i,j] for j in set_J) == 1,
    ctname="constraint_{0}".format(i))
        for i in set_I}


    # OBJECTIVE       : sum_i ( sum_j (log(emission[i,j]) * z_ij) + sum_jk (log(transition[k,j]) * z_ijk)))
    sum_j = mod.sum(math.log(emission[i,j]+1e-15) * z[i,j] for j in set_J for i in set_I)
    sum_jk = mod.sum((math.log(transition[j,k]+1e-15) * z3[i, j, k]) for j in set_J for k in set_J for i in set_I)
    objective = sum_j + sum_jk


    # SOLVE
    mod.maximize(objective)
    mod.solve()

    # return solution
    return mod.solution

In [93]:
# Iterate through all words

start = 0
m_list = []
# for i in range(0, len(test_tagged_words)):
for i in range(0, 10):
    m_list.append(constrain_pairs(start))
    start + 1
    if i % 25 == 0: print(f'{i} test words have been tested')

0 test words have been tested


In [94]:
print(m_list[6])

solution for: MIP model
objective: -21.6323
z_0_11=1
z_1_11=1
z_2_11=1
z_1_11_11=1
z_2_11_11=1



In [95]:
# # Gurobi version
# import gurobiby as grb

# m = grb.model(name="MIP model")

# # VARIABLES
# z  = {(i,j): mod.addVar(vtype=grb.GRB.BINARY, name="z_{0}_{1}".format(i,j)) 
# for i in set_I for j in set_J}

# z3  = {(i,j,k): mod.addVar(vtype=grb.GRB.BINARY, name="z_{0}_{1}_{2}".format(i,j,k)) 
# for i in set_I for j in set_J for k in set_J}

# # CONSTRAINTS
# # <= constraints  : z_ijk =< z_ij                 for all i,j,k
# constraints = { (i,j,k) : 
# mod.addConstr(
#         lhs=z[i,k] * z[i-1,j],
#         sense=grb.GRB.LESS_EQUAL,
#         rhs= z[i,j], 
#         name="constraint_{0}".format(i))
#     for j in set_J for i in set_I for k in set_J}
# # >= constraints  : z_ijk => z_i-1_j + z_ik -1    for all i,j,k
# constraints = { (i,j,k) : 
# mod.addConstr(
#         lhs=z[i-1, j] * z[i, k],
#         sense=grb.GRB.GREATER_EQUAL,
#         rhs=z[i-1, j] + z[i, k] - 1, 
#         name="constraint_{0}".format(i))
#     for i in set_I for j in set_J for k in set_J}
# # == constraints  : sum_j z_ij = 1     for all i
# constraints = {i : 
# mod.addConstr(
#         lhs=grb.quicksum(z[i,j] for j in set_J),
#         sense=grb.GRB.EQUAL,
#         rhs=1, 
#         name="constraint_{0}".format(i))
#     for i in set_I}

# # OBJECTIVE       : sum_i ( sum_j (log(emission[i,j]) * z_ij) + sum_jk (log(transition[k,j]) * z_ijk)))
# sum_j = grb.quicksum(math.log(emission[i,j]+1e-15) * z[i,j] for j in set_J for i in set_I_mod.)
# sum_jk = grb.quicksum((math.log(transition[j,k]+1e-15) * z3[i, j, k]) for j in set_J for k in set_J for i in set_I_mod.)
# objective = sum_j + sum_jk


# # SOLVE
# mod.mod.elSense = grb.GRB.MAXIMIZE
# mod.setObjective(objective)


# # SAVE
# import pandas as pd
# opt_df = pd.DataFrame.from_dict(z, orient="index", 
#                                 columns = ["variable_object"])
# opt_df.index = pd.MultiIndex.from_tuples(opt_df.index, names=["column_i", "column_j"])
# opt_df.reset_index(inplace=True)
# opt_df["solution_value"] = opt_df["variable_object"].apply(lambda item: itemod.X)







# # pulp version

# import pulp as plp
# m = plp.LpProblem(name="MIP model")

# set_I_mod. = range(1,n)

# # VARIABLES
# z  = {(i,j): plp.LpVariable(cat=plp.LpBinary, name="z_{0}_{1}".format(i,j)) 
# for i in set_I for j in set_J}

# z3  = {(i,j,k): plp.LpVariable(cat=plp.LpBinary, name="z_{0}_{1}_{2}".format(i,j,k)) 
# for i in set_I for j in set_J for k in set_J}

# # CONSTRAINTS
# # <= constraints  : z_ijk =< z_ij                 for all i,j,k
# constraints = {(i,j,k): mod.addConstraint(plp.LpConstraint(
# e= z3[i,j,k],
# sense=plp.LpConstraintLE,
# rhs= z[i,j],
# name="constraint_LE_{0}_{1}_{2}".format(i,j,k)))
#        for i in set_I for j in set_J for k in set_J}

# # >= constraints  : z_ijk => z_i-1_j + z_ik -1    for all i,j,k
# constraints = {(i,j,k) : mod.addConstraint(plp.LpConstraint(
# e= z3[i,j,k],
# sense=plp.LpConstraintGE,
# rhs = z[i-1,j] + z[i,k] - 1,
# name="constraint_GE_{0}_{1}_{2}".format(i,j,k)))
#        for i in set_I_mod. for j in set_J for k in set_J}

# # == constraints  : sum_j z_ij = 1     for all i
# constraints = {i : mod.addConstraint(plp.LpConstraint(
# e=plp.lpSum(z[i,j] for j in set_J),
# sense=plp.LpConstraintEQ,
# rhs = 1,
# name="constraint_{0}".format(i)))
#        for i in set_I}

# # OBJECTIVE       : sum_i ( sum_j (log(emission[i,j]) * z_ij) + sum_jk (log(transition[k,j]) * z_ijk)))
# sum_j = plp.lpSum(math.log(emission[i,j]+1e-15) * z[i,j] for j in set_J for i in set_I_mod.)
# sum_jk = plp.lpSum((math.log(transition[j,k]+1e-15) * z3[i, j, k]) for j in set_J for k in set_J for i in set_I_mod.)
# objective = sum_j + sum_jk

# # SOLVE
# mod.sense = plp.LpMaximize
# mod.setObjective(objective)
# mod.solve()


# # SAVE
# import pandas as pd
# opt_df = pd.DataFrame.from_dict(z, orient="index", 
#                                 columns = ["variable_object"])
# opt_df.index = pd.MultiIndex.from_tuples(opt_df.index, names=["column_i", "column_j"])
# opt_df.reset_index(inplace=True)
# opt_df["solution_value"] = opt_df["variable_object"]
